In [1]:
import joblib
import gzip
import json
import numpy as np
import random
import pandas as pd
import os
from sqlalchemy import create_engine
import urllib
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

In [2]:
## Login to Azure Key Vault

# replace 'my-key-vault-url' with the URL of your Azure Key Vault
vault_url = 'https://kv-colmex.vault.azure.net/'

# DefaultAzureCredential will use the credentials of your logged-in Azure account
credential = DefaultAzureCredential()

# create a SecretClient
client = SecretClient(vault_url=vault_url, credential=credential)

In [3]:
list_reviews = joblib.load("../extract/downloaded_reviews.joblib")

In [4]:
ejemplo = list_reviews[0]

In [5]:
with gzip.open(os.path.join("../extract/",ejemplo), 'r') as fin:
    decompressed_data = fin.read().decode('utf-8')
json_objects = [json.loads(line) for line in decompressed_data.splitlines() if line]
df = pd.DataFrame(json_objects)

In [6]:
df = df.sample(10)

In [7]:
df = df.reset_index().rename(columns={"index": "reviewID"}).drop(columns=["image", "style"])

In [20]:
# parameters
server = client.get_secret('db-server').value
database = client.get_secret('database').value
username = client.get_secret('db-username').value
password = client.get_secret('db-password').value
driver = '{ODBC Driver 17 for SQL Server}'
table = 'reviews_test'

# create the connection string
params = urllib.parse.quote_plus(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')

engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

df.to_sql(f'{table}', con=engine, if_exists='append', index=False)

10